In [13]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# !pip install gekko==0.2.6
# !pip install keplergl

In [15]:
import os
import json
import pandas as pd
import numpy as np
import re 

from matplotlib import pyplot as plt 
# from keplergl import KeplerGl

## Functions

In [16]:
def exists_in_population(pop_dict, name):
  """
  Input: population dictionary, poi name string
  Output: boolean
  Function returns boolean indicating whether poi name entered exists in 
  population dictionary, used in conjunction with get_poi_population
  """
  result = []
  for poi_name in pop_dict.keys():
    if (re.search(f'{name}', poi_name)):
      result.append(pop_dict[poi_name])
  if not result:
    return False
  else:
    return True

def get_poi_population(pop_dict, name):
  """
  Input: population dictionary, poi name string
  Output: poi population integer
  Function returns the population for the point of interest in question, if found.
  """
  if not exists_in_population(pop_dict, name):
    return 0
  if exists_in_population(pop_dict, name):
    for poi_name in pop_dict.keys():
      if (re.search(f'{name}', poi_name)):
        return pop_dict[poi_name]

## usc model

In [17]:
def calculating_risk_for_single_community(vec_I,population,sigma,ave_k):
  """ This function computes the daily risk score for a single community given the number of daily infected cases, population of the community, 
  inverse of average recovery dates, and number of times actual infected cases is higher than reported ones.
  """

# Inputs:
# - vec_I       :   Represents the vector of number of daily infected cases reported for the community
# - population  :   Indicates the population of the community
# - sigma       :   Is equal to 1/D where D represents the average recovery days (usually D is around 7.5)
# - ave_k       :   Average value for the ration of actual active daily infected cases to reported ones (should be >=1)

# Outputs:
# - risk        :    Vector of the risk score of the community over time
  c = len(vec_I)
  matrix_I = vec_I[np.newaxis,:]
  beta_SIR, R, risk = np.zeros((c-1,)), np.zeros((c-1,)), np.zeros((c-1,))
  for time in range(c-1):
    # clear_output(wait=True)
    next_I, curr_I, N = ave_k*vec_I[time+1], ave_k*vec_I[time], int(population) # FA: made population to int(population)

    if curr_I != 0:
        a1,a2 = curr_I-(curr_I**2)/N, next_I-(next_I**2)/N
        b1,b2 = -sigma*curr_I, -sigma*next_I
        if next_I>curr_I:
            delta_t =1
        else:
            delta_t =-1
        A = a1*a2*delta_t
        B = (a1*b2+a2*b1)*delta_t-(a1-a2)
        C = b1*b2*delta_t-(b1-b2)
        if B.item()**2-4*A.item()*C.item() > 0:
            output = (-B+np.sqrt(B**2-4*A*C))/(2*A)
        else:
            output = 0
        #output = (next_I - curr_I+sigma*curr_I)/(curr_I-(1/N)*curr_I**2)
    else:
        output = 0
    # print("curr", curr_I, "next", next_I) # comment/uncomment
    beta_SIR[time] = max(0,output) 
    # beta_SIR[time] = max(0,solve_beta_for_single_time_exponential(matrix_I[0,time+1],matrix_I[0,time],sigma,int(population),0) )
    R[time] = beta_SIR[time] / sigma
    risk[time] = max((5000)*R[time]*vec_I[time]*ave_k/(1.0*int(population)),0) # FA: made population to int(population), made 10,000 to 5,000
    # risk[time] = max((10000)*R[time]*vec_I[time]*ave_k/(1.0*int(population)),0) # FA: made population to int(population)
    # clear_output(wait=True) 

  return risk

## Pull in needed files:
 - LA point of interest cases over time
 - LA point of interest populations (average prior month; safegraph) 

In [18]:
os.chdir('/content/drive/My Drive/safegraph_data')
f = open('lacounty_covid_poi.json')
lacounty_covid_poi = json.load(f)

os.chdir('safegraph_monthly_data')
f = open('POI_population.json')
population_poi = json.load(f)

### Create list of unique POIs to loop through

In [19]:
poi_list = []
for i in population_poi.keys():
  poi_list.append(i)

### Find matched POIs, assign a risk assessment to their cases timeline, and store this in a dictionary
#### POIs with with 0s across days indicate no new covid19 cases added to the previous day count

In [20]:
cases_list = []
get_poi_name = ''
poi_population = ''
poi_dict = {}

for POI_Name in poi_list:
  for key, val in lacounty_covid_poi.items():
    for poi in val:
      if (re.search(f'^{POI_Name}$', poi[0])):
        get_poi_name = POI_Name
        # print(str(key) + ' ' + str(POI_Name) + ' ==== ' + str(poi[1]))
        cases_list.append(int(poi[1]))
        poi_population = get_poi_population(population_poi, POI_Name)
  if (len(get_poi_name)!=0):
    risk_values = calculating_risk_for_single_community(vec_I=np.array(cases_list),
                                                        population=poi_population,
                                                        sigma=1/7.5,
                                                        ave_k=12)
    # f.write(f"{str(get_poi_name)}: {str(risk_values)}\n")
    print(f'{str(get_poi_name)}: {str(risk_values)}')
    poi_dict[str(get_poi_name)] = np.array(risk_values).tolist()
    cases_list = []
    get_poi_name = ''


Pomona Valley Hospital Medical Center: [78.74015636]
Food 4 Less: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Men's Central Jail: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Hollywood Presbyterian Medical Center: [0. 0.]
Super King Markets: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
McDonald's: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Forever 21: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Centinela Hospital Medical Center: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Subway: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Goodwill Industries: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Lexus of Cerritos: [1392.5104697   500.24358337 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


Antelope Valley Care Center: [   0.         1285.36352083    0.            0.                   inf
    0.                   inf    0.            0.            0.
    0.            0.            0.         4613.50718851  180.44904692
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.                   inf    0.
    0.            0.            0.            0.            0.
    0.         1194.01699282    0.            0.            0.
    0.            0.            0.            0.        ]
Harbor Freight Tools: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Western Bagel: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Shutters On The Beach: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Tarzana Treatment Center: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

### Store this dictionary in a JSON file

In [21]:
os.chdir('/content/drive/My Drive/safegraph_data/')
with open('poi_risk_values.json', 'w') as json_file:
  json.dump(poi_dict, json_file)


## Load the output from a JSON file

In [22]:
# os.chdir('/content/drive/My Drive/safegraph_data')
# f = open("poi_risk_values.json")
# data = json.load(f)

# for key,val in data.items():
#     print(str(key) + ' ' + str(val))


### Visuals and plots of results?

In [23]:
# # data


# y = []
# for key, val in lacounty_covid_poi.items():
#   for item in val:
#     # print(item[1])
#     if (re.search('^Windsor Gardens Convalescent Center of Hawthorne', item[0])):
#       print(str(key) + ' ' + str(item[0]) + ' --cases: ' + str(item[1]))
#       y.append(item[1])


# x = np.linspace(start=17, stop=60, num=44)

# # x.shape
# y = np.array(y)

# plt.plot(x, y)
# plt.show()


In [24]:

# os.chdir('/content/drive/My Drive/safegraph_data/End_Of_Day_Data__Covid19_CSV')
# dod_df = pd.read_csv('Ending_Dec20_2020_day_delta.csv')



# df1_map = KeplerGl(height=600, width=800)
# df1_map
# # df1_map.save_to_html(file_name='Non-Residential_Settings.html')


In [25]:
# df1_map.add_data(data=dod_df, name='POI Cases in Los Angeles')